In [1]:
import numpy as np

In [2]:
from tdse.matrix_py import mat_vec_mul_tridiag as forward
from tdse.matrix_py import gaussian_elimination_tridiagonal as backward

In [3]:
N = 10
a = np.ones(N)
b = np.ones(N-1)
c = np.zeros(N-1)

In [4]:
v = np.arange(N)
v

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
vv = forward(a,b,c,v)
vv

array([ 0,  1,  3,  5,  7,  9, 11, 13, 15, 17])

In [6]:
vvv = backward(a,b,c,vv)
vvv

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])